In [ ]:
import pandas as pd
import numpy as np

import os
import sys

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import  accuracy_score,classification_report

In [ ]:
Emotions = pd.read_csv('emotion.csv')

In [ ]:
Emotions=Emotions.fillna(0)
print(Emotions.isna().any())
Emotions.shape

In [ ]:
Y=Emotions['Emotions']
X= Emotions.drop('Emotions',axis= 1)

In [ ]:
Y_class=Y.replace({'neutral':0,'happy':1,'sad':2,'angry':3, 'fear':4,'disgust':5,'surprise':6})

In [ ]:
#train test split
x_train, x_test, y_train, y_test = train_test_split(X, Y_class, random_state=35,test_size=0.2, shuffle=True)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
# scale data
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
from catboost import CatBoostClassifier,Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score


# 创建 CatBoost 分类器模型
catboost_model = CatBoostClassifier(iterations=1000,
                                    learning_rate=0.3,
                                    depth=5,
                                    random_state=42,
                                    l2_leaf_reg=5,
                                    loss_function='MultiClass',  
                                    classes_count=7,  
                                    )
train_pool = Pool(x_train, label=y_train)
test_pool = Pool(x_test, label=y_test)

# 训练 CatBoost 模型
catboost_model.fit(train_pool, eval_set=test_pool,plot=True)

In [ ]:
#训练集
y_pred_train = catboost_model.predict(train_pool)

In [ ]:
accuracy= accuracy_score(y_train, y_pred_train)
print(f"train Accuracy: {accuracy}")

In [ ]:
# 测试集上概率
y_pred = catboost_model.predict(test_pool)

In [ ]:
accuracy= accuracy_score(y_test, y_pred)
print(f"test Accuracy: {accuracy}")

In [ ]:
# 计算混淆矩阵
conf_matrix = confusion_matrix(y_test, y_pred)
emotions=['neutral','happy','sad','angry', 'fear','disgust','surprise']
# 使用seaborn绘制热力图
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=[i for i in emotions],
            yticklabels=[i for i in emotions])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()